In [38]:
# Import basic libraries
import pandas as pd
import numpy as np


In [39]:
# The training dataset comprises 343746 rows and 21 columns
taxi = pd.read_csv("https://datasciencebocconi.github.io/Data/taxi_train.csv")

# The training dataset comprises 147546 rows and 19 columns (the label `response` is not available)
taxi_pred = pd.read_csv("https://datasciencebocconi.github.io/Data/taxi_test.csv")

# Let's have a look at the training dataset
taxi

,weight,response,pickup_hour,pickup_month,pickup_week,pickup_doy,pickup_wday,length_time,pickup_BoroCode,pickup_NTACode,...,dropoff_NTACode,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,vendor_id,passenger_count,trip_distance,fare_amount,pair
0,1,1.75,18,5,19,132,4,520,1,MN15,...,MN19,-73.983009,40.766602,-73.967972,40.760677,1,1,0.90,7.0,MN15-MN19
1,1,2.95,18,5,19,127,6,824,1,MN21,...,MN24,-73.989510,40.734470,-73.997185,40.717976,1,1,1.40,10.0,MN21-MN24
2,1,2.46,7,5,19,132,4,1016,1,MN25,...,MN27,-74.012741,40.702229,-73.998878,40.713711,2,1,1.14,11.5,MN25-MN27
3,1,1.66,11,5,20,134,6,550,1,MN99,...,MN20,-73.970909,40.767231,-73.979507,40.752331,2,1,1.28,7.5,MN99-MN20
4,1,1.75,20,5,19,128,7,501,1,MN20,...,MN22,-73.973869,40.747925,-73.986618,40.730328,1,1,1.40,7.5,MN20-MN22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343741,1,2.00,20,5,21,145,3,740,1,MN12,...,MN36,-73.978676,40.783512,-73.938370,40.850533,2,1,5.84,18.5,MN12-MN36
343742,1,1.00,23,5,21,141,6,193,1,MN14,...,MN12,-73.982048,40.775452,-73.976860,40.783520,2,6,0.70,4.5,MN14-MN12
343743,1,2.35,19,5,19,132,4,764,1,MN20,...,MN22,-73.974472,40.747253,-73.983070,40.726955,1,1,1.70,10.0,MN20-MN22
343744,1,2.65,21,5,20,140,5,955,1,MN13,...,MN27,-73.993607,40.745293,-73.987350,40.722256,1,1,2.20,12.0,MN13-MN27


In [40]:
dirty = taxi.copy()

In [41]:
from sklearn.preprocessing import TargetEncoder

In [42]:
dirty.drop(columns=['weight','pair','pickup_month'],inplace=True)

In [43]:
categories = dirty[['pickup_NTACode','dropoff_NTACode','pickup_doy']]


In [44]:
dirty_train = dirty.response

In [45]:
encoder = TargetEncoder(target_type = 'continuous', smooth = 100)
X_train_encoded = encoder.fit_transform(categories, dirty_train)


In [46]:
X_train_encoded

array([[2.53655207, 2.49642731, 2.82934101],
       [2.34386349, 2.59790021, 2.84498875],
       [3.43956597, 2.58474031, 2.82934101],
       ...,
       [2.42059859, 2.34244209, 2.84327726],
       [2.40943433, 2.5653076 , 3.03113523],
       [2.32584855, 2.38219787, 2.65724819]])

In [47]:
dirty[['pickup_NTACode','dropoff_NTACode','pickup_doy']] = X_train_encoded

In [48]:
dirty.pickup_doy

0         2.829341
1         2.844989
2         2.829341
3         2.998324
4         2.547331
            ...   
343741    2.835090
343742    2.873706
343743    2.843277
343744    3.031135
343745    2.657248
Name: pickup_doy, Length: 343746, dtype: float64

In [49]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [50]:
dirty = encode(dirty,'pickup_hour',23)

In [51]:
dirty = encode(dirty, 'pickup_wday',7)

In [52]:
dirty.head()

,response,pickup_hour,pickup_week,pickup_doy,pickup_wday,length_time,pickup_BoroCode,pickup_NTACode,dropoff_BoroCode,dropoff_NTACode,...,dropoff_longitude,dropoff_latitude,vendor_id,passenger_count,trip_distance,fare_amount,pickup_hour_sin,pickup_hour_cos,pickup_wday_sin,pickup_wday_cos
0,1.75,18,19,2.829341,4,520,1,2.536552,1,2.496427,...,-73.967972,40.760677,1,1,0.90,7.0,-0.979084,0.203456,-4.338837e-01,-0.900969
1,2.95,18,19,2.844989,6,824,1,2.343863,1,2.597900,...,-73.997185,40.717976,1,1,1.40,10.0,-0.979084,0.203456,-7.818315e-01,0.623490
2,2.46,7,19,2.829341,4,1016,1,3.439566,1,2.584740,...,-73.998878,40.713711,2,1,1.14,11.5,0.942261,-0.334880,-4.338837e-01,-0.900969
3,1.66,11,20,2.998324,6,550,1,2.547786,1,2.309413,...,-73.979507,40.752331,2,1,1.28,7.5,0.136167,-0.990686,-7.818315e-01,0.623490
4,1.75,20,19,2.547331,7,501,1,2.429523,1,2.336846,...,-73.986618,40.730328,1,1,1.40,7.5,-0.730836,0.682553,-2.449294e-16,1.000000


In [53]:
dirty_encoded = dirty.drop(columns=['pickup_hour','pickup_wday'])

In [54]:
dirty_encoded.head()

,response,pickup_week,pickup_doy,length_time,pickup_BoroCode,pickup_NTACode,dropoff_BoroCode,dropoff_NTACode,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,vendor_id,passenger_count,trip_distance,fare_amount,pickup_hour_sin,pickup_hour_cos,pickup_wday_sin,pickup_wday_cos
0,1.75,19,2.829341,520,1,2.536552,1,2.496427,-73.983009,40.766602,-73.967972,40.760677,1,1,0.90,7.0,-0.979084,0.203456,-4.338837e-01,-0.900969
1,2.95,19,2.844989,824,1,2.343863,1,2.597900,-73.989510,40.734470,-73.997185,40.717976,1,1,1.40,10.0,-0.979084,0.203456,-7.818315e-01,0.623490
2,2.46,19,2.829341,1016,1,3.439566,1,2.584740,-74.012741,40.702229,-73.998878,40.713711,2,1,1.14,11.5,0.942261,-0.334880,-4.338837e-01,-0.900969
3,1.66,20,2.998324,550,1,2.547786,1,2.309413,-73.970909,40.767231,-73.979507,40.752331,2,1,1.28,7.5,0.136167,-0.990686,-7.818315e-01,0.623490
4,1.75,19,2.547331,501,1,2.429523,1,2.336846,-73.973869,40.747925,-73.986618,40.730328,1,1,1.40,7.5,-0.730836,0.682553,-2.449294e-16,1.000000


In [55]:
dirty_encoded = pd.get_dummies(dirty_encoded)

In [56]:
dirty_encoded.head()

,response,pickup_week,pickup_doy,length_time,pickup_BoroCode,pickup_NTACode,dropoff_BoroCode,dropoff_NTACode,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,vendor_id,passenger_count,trip_distance,fare_amount,pickup_hour_sin,pickup_hour_cos,pickup_wday_sin,pickup_wday_cos
0,1.75,19,2.829341,520,1,2.536552,1,2.496427,-73.983009,40.766602,-73.967972,40.760677,1,1,0.90,7.0,-0.979084,0.203456,-4.338837e-01,-0.900969
1,2.95,19,2.844989,824,1,2.343863,1,2.597900,-73.989510,40.734470,-73.997185,40.717976,1,1,1.40,10.0,-0.979084,0.203456,-7.818315e-01,0.623490
2,2.46,19,2.829341,1016,1,3.439566,1,2.584740,-74.012741,40.702229,-73.998878,40.713711,2,1,1.14,11.5,0.942261,-0.334880,-4.338837e-01,-0.900969
3,1.66,20,2.998324,550,1,2.547786,1,2.309413,-73.970909,40.767231,-73.979507,40.752331,2,1,1.28,7.5,0.136167,-0.990686,-7.818315e-01,0.623490
4,1.75,19,2.547331,501,1,2.429523,1,2.336846,-73.973869,40.747925,-73.986618,40.730328,1,1,1.40,7.5,-0.730836,0.682553,-2.449294e-16,1.000000


In [57]:
dirty_encoded_dummies = pd.get_dummies(dirty_encoded,columns=['pickup_BoroCode','pickup_week','dropoff_BoroCode','vendor_id','passenger_count'],dtype='Int64')

In [58]:

dirty_encoded_dummies

,response,pickup_doy,length_time,pickup_NTACode,dropoff_NTACode,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance,...,vendor_id_1,vendor_id_2,passenger_count_0,passenger_count_1,passenger_count_2,passenger_count_3,passenger_count_4,passenger_count_5,passenger_count_6,passenger_count_8
0,1.75,2.829341,520,2.536552,2.496427,-73.983009,40.766602,-73.967972,40.760677,0.90,...,1,0,0,1,0,0,0,0,0,0
1,2.95,2.844989,824,2.343863,2.597900,-73.989510,40.734470,-73.997185,40.717976,1.40,...,1,0,0,1,0,0,0,0,0,0
2,2.46,2.829341,1016,3.439566,2.584740,-74.012741,40.702229,-73.998878,40.713711,1.14,...,0,1,0,1,0,0,0,0,0,0
3,1.66,2.998324,550,2.547786,2.309413,-73.970909,40.767231,-73.979507,40.752331,1.28,...,0,1,0,1,0,0,0,0,0,0
4,1.75,2.547331,501,2.429523,2.336846,-73.973869,40.747925,-73.986618,40.730328,1.40,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343741,2.00,2.835090,740,2.326441,4.274101,-73.978676,40.783512,-73.938370,40.850533,5.84,...,0,1,0,1,0,0,0,0,0,0
343742,1.00,2.873706,193,2.386222,2.362675,-73.982048,40.775452,-73.976860,40.783520,0.70,...,0,1,0,0,0,0,0,0,1,0
343743,2.35,2.843277,764,2.420599,2.342442,-73.974472,40.747253,-73.983070,40.726955,1.70,...,1,0,0,1,0,0,0,0,0,0
343744,2.65,3.031135,955,2.409434,2.565308,-73.993607,40.745293,-73.987350,40.722256,2.20,...,1,0,0,1,0,0,0,0,0,0


In [59]:
Final_not_normalized = dirty_encoded_dummies.drop(columns=['pickup_longitude','pickup_latitude','dropoff_latitude','dropoff_longitude'])

In [60]:
Final_not_normalized_response = Final_not_normalized.response


In [61]:
Final_not_normalized.drop(columns='response',inplace=True)

In [62]:
Final_not_normalized


,pickup_doy,length_time,pickup_NTACode,dropoff_NTACode,trip_distance,fare_amount,pickup_hour_sin,pickup_hour_cos,pickup_wday_sin,pickup_wday_cos,...,vendor_id_1,vendor_id_2,passenger_count_0,passenger_count_1,passenger_count_2,passenger_count_3,passenger_count_4,passenger_count_5,passenger_count_6,passenger_count_8
0,2.829341,520,2.536552,2.496427,0.90,7.0,-9.790841e-01,0.203456,-4.338837e-01,-0.900969,...,1,0,0,1,0,0,0,0,0,0
1,2.844989,824,2.343863,2.597900,1.40,10.0,-9.790841e-01,0.203456,-7.818315e-01,0.623490,...,1,0,0,1,0,0,0,0,0,0
2,2.829341,1016,3.439566,2.584740,1.14,11.5,9.422609e-01,-0.334880,-4.338837e-01,-0.900969,...,0,1,0,1,0,0,0,0,0,0
3,2.998324,550,2.547786,2.309413,1.28,7.5,1.361666e-01,-0.990686,-7.818315e-01,0.623490,...,0,1,0,1,0,0,0,0,0,0
4,2.547331,501,2.429523,2.336846,1.40,7.5,-7.308360e-01,0.682553,-2.449294e-16,1.000000,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343741,2.835090,740,2.326441,4.274101,5.84,18.5,-7.308360e-01,0.682553,4.338837e-01,-0.900969,...,0,1,0,1,0,0,0,0,0,0
343742,2.873706,193,2.386222,2.362675,0.70,4.5,-2.449294e-16,1.000000,-7.818315e-01,0.623490,...,0,1,0,0,0,0,0,0,1,0
343743,2.843277,764,2.420599,2.342442,1.70,10.0,-8.878852e-01,0.460065,-4.338837e-01,-0.900969,...,1,0,0,1,0,0,0,0,0,0
343744,3.031135,955,2.409434,2.565308,2.20,12.0,-5.195840e-01,0.854419,-9.749279e-01,-0.222521,...,1,0,0,1,0,0,0,0,0,0


In [63]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(criterion='poisson',max_depth=50,n_jobs=8,random_state=42)


In [64]:
regr.fit(Final_not_normalized,Final_not_normalized_response)

RandomForestRegressor(criterion='poisson', max_depth=50, n_jobs=8,
                      random_state=42)

In [65]:
y_pred = regr.predict(Final_not_normalized)

In [66]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred,Final_not_normalized_response)

0.24814953460910075

In [70]:
rf_coef = pd.Series(regr.feature_importances_, index = Final_not_normalized.columns)
rf_coef.sort_values(ascending = False).head(15)

fare_amount          0.759211
length_time          0.046340
trip_distance        0.035238
dropoff_NTACode      0.030248
pickup_doy           0.028129
pickup_NTACode       0.027487
pickup_hour_sin      0.014725
pickup_hour_cos      0.012026
pickup_wday_sin      0.008049
pickup_wday_cos      0.004633
pickup_week_19       0.002803
pickup_week_21       0.002759
passenger_count_2    0.002667
pickup_week_20       0.002431
passenger_count_1    0.002369
dtype: float64

In [67]:
mean_pred = Final_not_normalized_response

In [78]:
mean_pred.response = np.median(Final_not_normalized_response)

In [81]:
mean_absolute_error(mean_pred,Final_not_normalized_response)

0.0

In [73]:

from sklearn.linear_model import LassoLarsCV


In [74]:
reg_Lasso = LassoLarsCV(cv=5,max_iter=1000,n_jobs=4).fit(Final_not_normalized, Final_not_normalized_response)

In [76]:
Lasso_predict = reg_Lasso.predict(Final_not_normalized)

In [77]:
mean_absolute_error(Lasso_predict,Final_not_normalized_response)

0.6146809436905204